In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from transformers import AutoTokenizer, BertForSequenceClassification

def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row['Sentence'])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words('english')]
        df_.loc[len(df_)] = {
            "index": row['index'],
            "Class": row['Class'],
            "Sentence": " ".join(filtered_sent)
        }
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv("amazon_cells_labelled.txt", delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data, columns)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )
    


<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:21: SyntaxWarning: invalid escape sequence '\w'
<>:22: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:21: SyntaxWarning: invalid escape sequence '\w'
<>:22: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Till\AppData\Local\Temp\ipykernel_11464\1349476086.py:20: SyntaxWarning: invalid escape sequence '\.'
  data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
C:\Users\Till\AppData\Local\Temp\ipykernel_11464\1349476086.py:21: SyntaxWarning: invalid escape sequence '\w'
  data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
C:\Users\Till\AppData\Local\Temp\ipykernel_11464\1349476086.py:22: SyntaxWarning: invalid escape sequence '\d'
  data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)          

In [2]:

    # # vectorize data using TFIDF and transform for PyTorch for scalability
    # word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
    # training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
    # training_data = training_data.todense()                             # convert to dense matrix for Pytorch
    # vocab_size = len(word_vectorizer.vocabulary_)
    # validation_data = word_vectorizer.transform(validation_data)
    # validation_data = validation_data.todense()
    # train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
    # train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
    # validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
    # validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()


In [ ]:
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')

# # Encode each sentence in training_data
# indexed_tokens = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in training_data]

Using cache found in C:\Users\Till/.cache\torch\hub\huggingface_pytorch-transformers_main


In [7]:
sequence_classification_model = torch.hub.load('huggingface/pytorch-transformers', 'modelForSequenceClassification', 'bert-base-cased-finetuned-mrpc')
sequence_classification_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased-finetuned-mrpc')
indexed_tokens = [sequence_classification_tokenizer.encode(sentence, add_special_tokens=True) for sentence in training_data]

Using cache found in C:\Users\Till/.cache\torch\hub\huggingface_pytorch-transformers_main
Using cache found in C:\Users\Till/.cache\torch\hub\huggingface_pytorch-transformers_main


In [17]:
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")

# Tokenize and encode the sentences into input IDs and attention masks
encoded_inputs = tokenizer(list(training_data), padding=True, truncation=True, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**encoded_inputs)
    logits = outputs.logits
    predicted_class_ids = logits.argmax(dim=1).tolist()

num_labels = len(model.config.id2label)

model = BertForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity", num_labels=num_labels)

labels = torch.tensor([1])

loss = model(**encoded_inputs, labels=labels).loss

ValueError: Expected input batch_size (900) to match target batch_size (1).

In [ ]:
print(sum(training_labels == predicted_class_ids)/len(training_labels))  # Print the predicted class IDs

[0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 